In [1]:
import torch

## Creating Layers

In [67]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.inputs = inputs
    self.output = torch.matmul(inputs, self.weights) + self.biases


## Activation Functions

### Sigmoid

In [68]:
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))

### Linear

In [69]:
class Activation_Linear:
    def forward(self, inputs):
        self.output = inputs

## Loss and Accuracy

In [70]:
class Accuracy:
    def compute(self, predictions, actuals):
        pred_labels = torch.argmax(predictions, dim=1)
        if len(actuals.shape) == 2:
            actuals = torch.argmax(actuals, dim=1)
        acc = torch.mean((pred_labels == actuals).float())
        return acc

def MeanSquaredError(actual, predicted):
    mse = torch.mean(0.5 * (actual - predicted) ** 2)
    return mse



### Preparing Dataset

In [79]:
# Create dataset
X = torch.rand(4,2)
y = torch.rand(4, 2)

In [80]:
hidden_layer_1 = DenseLayer(2, 4)
activation1 = Activation_Sigmoid()
output_layer = DenseLayer(4, 2)
activation2 = Activation_Linear()

In [81]:
def forward_pass(X):
  hidden_layer_1.forward(X)

  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)

  activation2.forward(output_layer.output)
  return activation2.output

In [82]:
def backward_propagation(fp):
    lr = 0.01  # Make sure learning rate is a scalar

    # Calculate errors and derivatives for output layer (assuming sigmoid activation)
    error_output = (fp - y) * fp * (1 - fp)  # Element-wise multiplication
    gradient_output = torch.matmul(activation1.output.T, error_output)

    # Update weights and biases for output layer
    output_layer.weights -= lr * gradient_output
    output_layer.biases -= lr * error_output.sum(axis=0)

    # Backpropagate the error to hidden layer
    error_hidden = torch.matmul(error_output, output_layer.weights.T) * activation1.output * (1 - activation1.output)
    gradient_hidden = torch.matmul(X.T, error_hidden)

    # Update weights and biases for hidden layer
    hidden_layer_1.weights -= lr * gradient_hidden
    hidden_layer_1.biases -= lr * error_hidden.sum(axis=0)

In [83]:
y_pred = forward_pass(X)
err = MeanSquaredError(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
epoch = 1000
for i in range(epoch):
  backward_propagation(y_pred)
  y_pred = forward_pass(X)
  err = MeanSquaredError(y, y_pred)
  if i%100==0:
    print("---------------------------------------------")
    print(f"epoch {i} loss:", err)
    print("prediction:",y_pred)
    print("Target value:",y)
    print("---------------------------------------------")

Initial loss: tensor(0.1504)
Initial prediction: tensor([[0.0061, 0.0145],
        [0.0061, 0.0145],
        [0.0061, 0.0145],
        [0.0061, 0.0145]])
---------------------------------------------
epoch 0 loss: tensor(0.1502)
prediction: tensor([[0.0063, 0.0150],
        [0.0063, 0.0150],
        [0.0063, 0.0150],
        [0.0063, 0.0150]])
Target value: tensor([[0.7704, 0.4369],
        [0.4038, 0.5769],
        [0.0436, 0.3361],
        [0.9852, 0.3400]])
---------------------------------------------
---------------------------------------------
epoch 100 loss: tensor(0.0754)
prediction: tensor([[0.2208, 0.1898],
        [0.2206, 0.1896],
        [0.2206, 0.1896],
        [0.2206, 0.1897]])
Target value: tensor([[0.7704, 0.4369],
        [0.4038, 0.5769],
        [0.0436, 0.3361],
        [0.9852, 0.3400]])
---------------------------------------------
---------------------------------------------
epoch 200 loss: tensor(0.0357)
prediction: tensor([[0.5018, 0.3804],
        [0.5011